In [1]:
%load_ext nb_black

In [2]:
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
import xgboost as xgb
from sklearn import metrics as m
from thundersvm import SVC as svmgpu
import calculateWeightUsingGa2 as aresult
import pandas as pd
import itertools
import random
from sklearn.utils import shuffle
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import VotingClassifier

randomseed = 42
np.random.seed(randomseed)

<IPython.core.display.Javascript object>

1. Read Dataset 
===

In [3]:
data = pd.read_csv("../dataset/seeds_dataset.txt", sep="\t", header=None)
data = shuffle(data)

le = LabelEncoder()
data.iloc[:, -1] = le.fit_transform(data.iloc[:, -1])
x = np.array(data.iloc[:, :-1])
y = np.array(data.iloc[:, -1])

print(np.unique(y))

xtrain, xtest, ytrain_original, ytest_original = train_test_split(
    x, y, test_size=0.3, random_state=10
)

ytrain = ytrain_original.copy()
ytest = ytest_original.copy()

[0 1 2]


<IPython.core.display.Javascript object>

In [4]:
# member values
clf = []
acc = []
finalacc = []
ypredproba_all = []
ypredconfprob_all = []

<IPython.core.display.Javascript object>

In [5]:
# orginal score using random forest classifier
rf = RandomForestClassifier(random_state=randomseed, n_estimators=10)
rf.fit(xtrain, ytrain)
print("original score", m.f1_score(ytest, rf.predict(xtest), average="weighted"))

original score 0.9524645173581343


<IPython.core.display.Javascript object>

In [6]:
def swapcolumns(trainval, testval, coldindexval):
    trainval[trainval != coldindexval] = 5
    testval[testval != coldindexval] = 5

    trainval[trainval == coldindexval] = 0
    trainval[trainval == 5] = 1

    testval[testval == coldindexval] = 0
    testval[testval == 5] = 1

    return trainval, testval


<IPython.core.display.Javascript object>

2. Generate baseline classifiers 
===

In [7]:
#================================================= 
# Class 0
# ===========================
ytrain = ytrain_original.copy()
ytest = ytest_original.copy()
ytrain,ytest= swapcolumns(ytrain,ytest,0)
#=================================================

rf=RandomForestClassifier(random_state=randomseed, n_estimators=100)
rf.fit(xtrain,ytrain)
rfpred=rf.predict(xtest)
print(m.f1_score(ytest,rfpred,average='weighted')) 

clf.append(rf)
acc.append(m.f1_score(ytest,rfpred,average='weighted'))
ypredproba_all.append(rf.predict_proba(xtest)) 

confmat=m.confusion_matrix(ytest,rfpred)
confsumh=np.sum(confmat,axis=0)
propconfmat=confmat.copy()
for i in range(propconfmat.shape[0]):
    propconfmat[:,i]= 100*propconfmat[:,i]/confsumh[i] 
ypredconfprob_all.append(propconfmat/100)


# =================================================
# classs 1
# =================================================
ytrain = ytrain_original.copy()
ytest = ytest_original.copy()
ytrain, ytest = swapcolumns(ytrain, ytest, 1)
# =================================================

rf = RandomForestClassifier(random_state=randomseed, n_estimators=100)
rf.fit(xtrain, ytrain)
rfpred = rf.predict(xtest)
print(m.f1_score(ytest, rfpred,average='weighted'))

clf.append(rf)
acc.append(m.f1_score(ytest, rfpred,average='weighted'))
ypredproba_all.append(rf.predict_proba(xtest))

confmat = m.confusion_matrix(ytest, rfpred)
confsumh = np.sum(confmat, axis=0)
propconfmat = confmat.copy()
for i in range(propconfmat.shape[0]):
    propconfmat[:, i] = 100 * propconfmat[:, i] / confsumh[i]
ypredconfprob_all.append(propconfmat / 100) 

#=================================================
# classs 2
#=================================================

ytrain=ytrain_original.copy()
ytest=ytest_original.copy()
ytrain,ytest= swapcolumns(ytrain,ytest,2)

#=================================================

rf=RandomForestClassifier(random_state=randomseed, n_estimators=100)
rf.fit(xtrain,ytrain)
rfpred=rf.predict(xtest)
print(m.f1_score(ytest,rfpred,average='weighted'))

clf.append(rf)
acc.append(m.f1_score(ytest,rfpred,average='weighted'))
ypredproba_all.append(rf.predict_proba(xtest))

confmat=m.confusion_matrix(ytest,rfpred)
confsumh=np.sum(confmat,axis=0)
propconfmat=confmat.copy()
for i in range(propconfmat.shape[0]):
    propconfmat[:,i]= 100*propconfmat[:,i]/confsumh[i] 
ypredconfprob_all.append(propconfmat/100)

# #=================================================



0.9359213250517598
0.9840147081526391
0.9686784791435955


<IPython.core.display.Javascript object>

In [8]:

import calculateWeightUsingGa2 as aresult
weightvalga=aresult.getbestvalues(acc)

finalcol = np.zeros((ytest.shape[0], 3))
finalcol[:, 0] = ypredproba_all[0][:, 0]*weightvalga[0]
finalcol[:, 1] = ypredproba_all[1][:, 0]*weightvalga[1]
finalcol[:, 2] = ypredproba_all[2][:, 0]*weightvalga[2]

finalpred = np.argmax(finalcol, axis=1)

ytest = ytest_original.copy()
print('trial1_scores f1_score',m.f1_score(ytest, finalpred,average='weighted')) 

# #=================================================

import calculateWeightUsingGa2 as aresult
weightvalga=aresult.getbestvalues([
    ypredconfprob_all[0][0][0] * ypredconfprob_all[1][1][1]*ypredconfprob_all[2][1][1],
    ypredconfprob_all[1][0][0] * ypredconfprob_all[0][1][1]*ypredconfprob_all[2][1][1],
    ypredconfprob_all[2][0][0] * ypredconfprob_all[0][1][1]*ypredconfprob_all[1][1][1]])

finalcol = np.zeros((ytest.shape[0], 3))
finalcol[:, 0] = ypredproba_all[0][:, 0]*weightvalga[0]
finalcol[:, 1] = ypredproba_all[1][:, 0]*weightvalga[1]
finalcol[:, 2] = ypredproba_all[2][:, 0]*weightvalga[2]

finalpred = np.argmax(finalcol, axis=1)

ytest = ytest_original.copy()
print('trial2_scores f1_score',m.f1_score(ytest, finalpred,average='weighted')) 

# #=================================================

import calculateWeightUsingGa2 as aresult
weightvalga=aresult.getbestvalues([
    ypredconfprob_all[0][0][0] * ypredconfprob_all[1][1][1]*ypredconfprob_all[2][1][1],
    ypredconfprob_all[1][0][0] * ypredconfprob_all[0][1][1]*ypredconfprob_all[2][1][1],
    ypredconfprob_all[2][0][0] * ypredconfprob_all[0][1][1]*ypredconfprob_all[1][1][1]])

finalcol = np.zeros((ytest.shape[0], 3))
finalcol[:, 0] = ypredproba_all[0][:, 0] * weightvalga[0] + ypredproba_all[0][:, 1] * ypredconfprob_all[0][0][1]
finalcol[:, 1] = ypredproba_all[1][:, 0] * weightvalga[1] + ypredproba_all[1][:, 1] * ypredconfprob_all[1][0][1]
finalcol[:, 2] = ypredproba_all[2][:, 0] * weightvalga[2] + ypredproba_all[2][:, 1] * ypredconfprob_all[2][0][1]

finalpred = np.argmax(finalcol, axis=1)

ytest = ytest_original.copy()
print('trial3_scores f1_score',m.f1_score(ytest, finalpred,average='weighted'))


# #=================================================

import calculateWeightUsingGa2 as aresult
weightvalga=aresult.getbestvalues([
    ypredconfprob_all[0][0][0] * ypredconfprob_all[1][1][1]*ypredconfprob_all[2][1][1] ,
    -ypredconfprob_all[0][0][1],

    ypredconfprob_all[1][0][0] * ypredconfprob_all[0][1][1]*ypredconfprob_all[2][1][1],
    -ypredconfprob_all[1][0][1],

    ypredconfprob_all[2][0][0] * ypredconfprob_all[0][1][1]*ypredconfprob_all[1][1][1],
    -ypredconfprob_all[2][0][1],

    ])

finalcol = np.zeros((ytest.shape[0], 3))
finalcol[:, 0] = ypredproba_all[0][:, 0]*weightvalga[0] + ypredproba_all[0][:, 1]*weightvalga[1]
finalcol[:, 1] = ypredproba_all[1][:, 0]*weightvalga[2] + ypredproba_all[1][:, 1]*weightvalga[3]
finalcol[:, 2] = ypredproba_all[2][:, 0]*weightvalga[4] + ypredproba_all[2][:, 1]*weightvalga[5]

finalpred = np.argmax(finalcol, axis=1)

ytest = ytest_original.copy()
print('trial4_scores f1_score',m.f1_score(ytest, finalpred,average='weighted'))


# #=================================================

import calculateWeightUsingGa2 as aresult
weightvalga=aresult.getbestvalues([
    ypredconfprob_all[0][0][0] ,
    ypredconfprob_all[1][0][0] ,
    ypredconfprob_all[2][0][0] ])

finalcol = np.zeros((ytest.shape[0], 3))
finalcol[:, 0] = weightvalga[0] * (ypredproba_all[0][:, 0] * ypredconfprob_all[0][0][0] + ypredproba_all[0][:, 1] * ypredconfprob_all[0][0][1])
finalcol[:, 1] = weightvalga[1] * (ypredproba_all[1][:, 0] * ypredconfprob_all[1][0][0] + ypredproba_all[1][:, 1] * ypredconfprob_all[1][0][1])
finalcol[:, 2] = weightvalga[2] * (ypredproba_all[2][:, 0] * ypredconfprob_all[2][0][0] + ypredproba_all[2][:, 1] * ypredconfprob_all[2][0][1])

finalpred = np.argmax(finalcol, axis=1)

ytest = ytest_original.copy()
print('trial5_scores f1_score',m.f1_score(ytest, finalpred,average='weighted')) 

# #=================================================

import calculateWeightUsingGa2 as aresult
weightvalga=aresult.getbestvalues([
    ypredconfprob_all[0][0][0] ,
    ypredconfprob_all[1][0][0] ,
    ypredconfprob_all[2][0][0] ])

finalcol = np.zeros((ytest.shape[0], 3))
finalcol[:, 0] =  (weightvalga[0] *ypredproba_all[0][:, 0] * ypredconfprob_all[0][0][0] + ypredproba_all[0][:, 1] * ypredconfprob_all[0][0][1])
finalcol[:, 1] = (weightvalga[1] * ypredproba_all[1][:, 0] * ypredconfprob_all[1][0][0] + ypredproba_all[1][:, 1] * ypredconfprob_all[1][0][1])
finalcol[:, 2] =  (weightvalga[2] *ypredproba_all[2][:, 0] * ypredconfprob_all[2][0][0] + ypredproba_all[2][:, 1] * ypredconfprob_all[2][0][1])

finalpred = np.argmax(finalcol, axis=1)

ytest = ytest_original.copy()
print('trial6_scores f1_score',m.f1_score(ytest, finalpred,average='weighted'))


0
1
2
3
4
5
6
7
8
9
2.791246486211413 [0.92789665 0.99643083 0.9731431 ]
trial1_scores f1_score 0.9524645173581343
0
1
2
3
4
5
6
7
8
9
2.585819629865763 [0.99767522 0.99716915 0.94099966]
trial2_scores f1_score 0.9524645173581343
0
1
2
3
4
5
6
7
8
9
2.5742378151865424 [0.98763281 0.98447785 0.94985462]
trial3_scores f1_score 0.9524645173581343
0
1
2
3
4
5
6
7
8
9
2.166325466735202 [0.99683099 0.65095806 0.98242069 0.77574334 0.93708429 0.92442428]
trial4_scores f1_score 0.8419171089807204
0
1
2
3
4
5
6
7
8
9
2.7399612825553126 [0.95826654 0.99671741 0.93120956]
trial5_scores f1_score 0.9524645173581343
0
1
2
3
4
5
6
7
8
9
2.780516781979524 [0.99216663 0.9961538  0.9387492 ]
trial6_scores f1_score 0.9524645173581343


<IPython.core.display.Javascript object>